In [2]:
import os
import numpy as np
import pandas as pd

# --- Detect working root path safely (works in Jupyter or as script) ---
try:
    # works when run as script
    root = os.path.abspath(os.path.join(os.path.dirname(__file__), ".."))
except NameError:
    # fallback for Jupyter (no __file__)
    root = os.path.abspath(os.path.join(os.getcwd(), ".."))


In [ ]:
# ---------------------------------------------------------------------
# 2. Define languages and file locations
# ---------------------------------------------------------------------
languages = ["Python", "MATLAB", "Julia_Float64", "Julia_BigFloat"]
file_map = {
    "Python": {
        "res": os.path.join(root, "python", "python_results.txt"),
        "time": os.path.join(root, "python", "python_time.txt")
    },
    "MATLAB": {
        "res": os.path.join(root, "matlab", "matlab_results.txt"),
        "time": os.path.join(root, "matlab", "matlab_time.txt")
    },
    "Julia_Float64": {
        "res": os.path.join(root, "julia", "julia_float_results.txt"),
        "time": os.path.join(root, "julia", "julia_float_time.txt")
    },
    "Julia_BigFloat": {
        "res": os.path.join(root, "julia", "julia_bigfloat_results.txt"),
        "time": os.path.join(root, "julia", "julia_bigfloat_time.txt")
    },
}

# ---------------------------------------------------------------------
# 3. Load potentials and runtimes
# ---------------------------------------------------------------------
potentials, runtimes = {}, {}
for lang, paths in file_map.items():
    try:
        if os.path.exists(paths["res"]):
            potentials[lang] = np.loadtxt(paths["res"])
        if os.path.exists(paths["time"]):
            with open(paths["time"]) as f:
                runtimes[lang] = float(f.read().strip())
    except Exception as e:
        print(f"Failed loading {lang}: {e}")

# ---------------------------------------------------------------------
# 4. Build comparison matrix
# ---------------------------------------------------------------------
N = len(languages)
matrix = np.full((N, N), np.nan)

n_points = 100000  # known number of test points in points.txt

for i, lang_i in enumerate(languages):
    for j, lang_j in enumerate(languages):
        if i == j:
            # Diagonal: average runtime per point (seconds)
            total_time = runtimes.get(lang_i, np.nan)
            matrix[i, j] = total_time / n_points if not np.isnan(total_time) else np.nan
        elif i < j:
            # Upper triangle: mean absolute numerical difference
            if lang_i in potentials and lang_j in potentials:
                p1, p2 = potentials[lang_i], potentials[lang_j]
                if len(p1) == len(p2):
                    diff = np.mean(np.abs(p1 - p2))
                    matrix[i, j] = diff
        # lower triangle left NaN intentionally

df = pd.DataFrame(matrix, index=languages, columns=languages, dtype=object)

# ---------------------------------------------------------------------
# 5. Consistent scientific formatting 
# ---------------------------------------------------------------------
def superscript_exp(exp):
    """Convert integer exponent to superscript string."""
    normal = "0123456789-+"
    super_ = "⁰¹²³⁴⁵⁶⁷⁸⁹⁻⁺"
    trans = str.maketrans("".join(normal), "".join(super_))
    return str(exp).translate(trans)

def sci_notation(x, unit=None):
    """Return consistent scientific notation with superscripts."""
    if np.isnan(x):
        return "–"
    exp = int(np.floor(np.log10(abs(x)))) if x != 0 else 0
    mant = x / 10**exp
    exp_str = superscript_exp(exp)
    formatted = f"{mant:05.2f}×10{exp_str}"
    if unit:
        formatted += f" {unit}"
    return formatted

# Create string-format DataFrame
df_fmt = pd.DataFrame("", index=languages, columns=languages)

for i in range(N):
    for j in range(N):
        val = df.iloc[i, j]
        if i == j:
            # Average time per point (seconds/point)
            df_fmt.iloc[i, j] = sci_notation(val, unit="s/pt")
        elif i < j:
            # Mean potential difference
            df_fmt.iloc[i, j] = sci_notation(val, unit=None)
        else:
            df_fmt.iloc[i, j] = "–"

# ---------------------------------------------------------------------
# 6. Save to text and CSV
# ---------------------------------------------------------------------
out_txt = os.path.join(root, "benchmark", "matrix_summary.txt")
out_csv = os.path.join(root, "benchmark", "matrix_summary.csv")

with open(out_txt, "w", encoding="utf-8") as f:
    f.write("=== Gravitational Potential Benchmark Matrix ===\n")
    f.write("Diagonal: Average Runtime per Point (s/pt)\n")
    f.write("Upper triangle: Mean |ΔPotential|\n")
    f.write("Lower triangle: –\n\n")
    f.write(df_fmt.to_string())
    f.write("\n")

df_fmt.to_csv(out_csv, index=True, encoding="utf-8")

print(df_fmt)
print(f"\n Matrix summary saved to:\n  {out_txt}\n  {out_csv}")


In [ ]:
# ============================================================
#  Gravitational Potential Benchmark Summary
#  Runtime and Accuracy vs. Julia BigFloat(250) Reference
# ============================================================
codes = ["Python", "MATLAB", "Julia_MP", "Julia_50"]

file_map = {
    "Python": {
        "res": os.path.join(root, "python", "python_results.txt"),
        "time": os.path.join(root, "python", "python_time.txt"),
    },
    "MATLAB": {
        "res": os.path.join(root, "matlab", "matlab_results.txt"),
        "time": os.path.join(root, "matlab", "matlab_time.txt"),
    },
    "Julia_MP": {
        "res": os.path.join(root, "julia", "julia_float_results.txt"),
        "time": os.path.join(root, "julia", "julia_float_time.txt"),
    },
    "Julia_50": {
        "res": os.path.join(root, "julia", "julia_bigfloat50_results.txt"),
        "time": os.path.join(root, "julia", "julia_bigfloat50_time.txt"),
    },
}

ref_path = os.path.join(root, "julia", "julia_bigfloat250_results.txt")
ref_time_path = os.path.join(root, "julia", "julia_bigfloat250_time.txt")

# ---------------------------------------------------------------------
# 2. Load reference (Julia 250-digit)
# ---------------------------------------------------------------------
if not os.path.exists(ref_path):
    raise FileNotFoundError("Reference file julia_bigfloat250_results.txt not found.")

U_ref = np.loadtxt(ref_path)
n_points = len(U_ref)

# ---------------------------------------------------------------------
# 3. Load results and runtimes
# ---------------------------------------------------------------------
potentials, runtimes = {}, {}

for code, paths in file_map.items():
    try:
        if os.path.exists(paths["res"]):
            potentials[code] = np.loadtxt(paths["res"])
        if os.path.exists(paths["time"]):
            with open(paths["time"]) as f:
                runtimes[code] = float(f.read().strip())
    except Exception as e:
        print(f"Failed loading {code}: {e}")

# ---------------------------------------------------------------------
# 4. Compute accuracy metrics
# ---------------------------------------------------------------------
rows = []
for code in codes:
    if code in potentials and len(potentials[code]) == n_points:
        diff = potentials[code] - U_ref
        mean_err = np.mean(np.abs(diff))
        rms_err = np.sqrt(np.mean(diff**2))
    else:
        mean_err = np.nan
        rms_err = np.nan

    total_time = runtimes.get(code, np.nan)
    time_per_point = total_time / n_points if not np.isnan(total_time) else np.nan

    rows.append((code, time_per_point, mean_err, rms_err))

df = pd.DataFrame(rows, columns=["Code", "runtime (s/pt)", "mean(error)", "RMS(error)"])

# ---------------------------------------------------------------------
# 5. Formatting utilities
# ---------------------------------------------------------------------
def superscript_exp(exp):
    normal = "0123456789-+"
    super_ = "⁰¹²³⁴⁵⁶⁷⁸⁹⁻⁺"
    return str(exp).translate(str.maketrans(normal, super_))

def sci_notation(x):
    if np.isnan(x):
        return "–"
    if x == 0:
        return "0"
    exp = int(np.floor(np.log10(abs(x))))
    mant = x / 10**exp
    return f"{mant:05.2f}×10{superscript_exp(exp)}"

df_fmt = df.copy()
for col in ["runtime (s/pt)", "mean(error)", "RMS(error)"]:
    df_fmt[col] = df[col].apply(sci_notation)

# ---------------------------------------------------------------------
# 6. Compute relative speedup (optional but useful)
# ---------------------------------------------------------------------
if "Julia_MP" in df and not np.isnan(df.loc[df["Code"]=="Julia_MP", "runtime (s/pt)"].values[0]):
    ref_speed = df.loc[df["Code"]=="Julia_MP", "runtime (s/pt)"].values[0]
    df_fmt["Speedup vs Julia_MP"] = df["runtime (s/pt)"].apply(
        lambda x: f"{ref_speed/x:5.2f}×" if not np.isnan(x) and x != 0 else "–"
    )

# ---------------------------------------------------------------------
# 7. Save results
# ---------------------------------------------------------------------
out_dir = os.path.join(root, "benchmark")
os.makedirs(out_dir, exist_ok=True)

out_txt = os.path.join(out_dir, "summary_table.txt")
out_csv = os.path.join(out_dir, "summary_table.csv")

with open(out_txt, "w", encoding="utf-8") as f:
    f.write("=== Gravitational Potential Accuracy & Speed Summary ===\n")
    f.write("Reference: Julia BigFloat (250 digits)\n\n")
    f.write(df_fmt.to_string(index=False))
    f.write("\n")

df_fmt.to_csv(out_csv, index=False, encoding="utf-8")

print(df_fmt)
print(f"\nSummary saved to:\n  {out_txt}\n  {out_csv}")


In [12]:
# ============================================================
#  Gravitational Potential Benchmark Summary
#  Runtime and Accuracy vs. Julia BigFloat(250) Reference
# ============================================================
from decimal import Decimal, localcontext

# ---------------------------------------------------------------------
# 1. Define root paths and files
# ---------------------------------------------------------------------

codes = ["Python", "MATLAB", "Julia_MP", "Julia_50"]

file_map = {
    "Python": {
        "res": os.path.join(root, "python", "python_results.txt"),
        "time": os.path.join(root, "python", "python_time.txt"),
    },
    "MATLAB": {
        "res": os.path.join(root, "matlab", "matlab_results.txt"),
        "time": os.path.join(root, "matlab", "matlab_time.txt"),
    },
    "Julia_MP": {
        "res": os.path.join(root, "julia", "julia_float_results.txt"),
        "time": os.path.join(root, "julia", "julia_float_time.txt"),
    },
    "Julia_50": {
        "res": os.path.join(root, "julia", "julia_bigfloat50_results.txt"),
        "time": os.path.join(root, "julia", "julia_bigfloat50_time.txt"),
    },
}

ref_path = os.path.join(root, "julia", "julia_bigfloat250_results.txt")

# ---------------------------------------------------------------------
# 2. Load reference (Julia 250)
# ---------------------------------------------------------------------
if not os.path.exists(ref_path):
    raise FileNotFoundError("Reference file julia_bigfloat250_results.txt not found.")

U_ref = np.loadtxt(ref_path, dtype=str)
n_points = len(U_ref)

# ---------------------------------------------------------------------
# 3. Load results and runtimes
# ---------------------------------------------------------------------
potentials, runtimes = {}, {}

for code, paths in file_map.items():
    try:
        if os.path.exists(paths["res"]):
            potentials[code] = np.loadtxt(paths["res"], dtype=str)
        if os.path.exists(paths["time"]):
            with open(paths["time"]) as f:
                runtimes[code] = float(f.read().strip())
    except Exception as e:
        print(f"Failed loading {code}: {e}")

# ---------------------------------------------------------------------
# 4. Compute accuracy metrics (true 50-digit precision comparison)
# ---------------------------------------------------------------------
rows = []
for code in codes:
    mean_err = rms_err = np.nan

    if code in potentials and len(potentials[code]) == n_points:
        if code == "Julia_50":
            diffs = []
            # Perform subtraction in a 50-digit local context
            for u50, uref in zip(potentials["Julia_50"], U_ref):
                try:
                    with localcontext() as ctx:
                        ctx.prec = 50
                        a = Decimal(u50)
                        b = Decimal(uref)
                        diff = abs(a - b)
                        diffs.append(diff)
                except Exception:
                    continue
            if diffs:
                mean_err = float(sum(diffs) / len(diffs))
                rms_err = float((sum((d ** 2 for d in diffs)) / len(diffs)).sqrt())
        else:
            # Normal float comparison for other codes
            p = np.array(potentials[code], dtype=np.float64)
            ref = np.array(U_ref, dtype=np.float64)
            diff = p - ref
            mean_err = np.mean(np.abs(diff))
            rms_err = np.sqrt(np.mean(diff**2))

    total_time = runtimes.get(code, np.nan)
    time_per_point = total_time / n_points if not np.isnan(total_time) else np.nan

    rows.append((code, time_per_point, mean_err, rms_err))

df = pd.DataFrame(rows, columns=["Code", "runtime (s/pt)", "mean(error)", "RMS(error)"])

# ---------------------------------------------------------------------
# 5. Formatting helpers
# ---------------------------------------------------------------------
def superscript_exp(exp):
    normal = "0123456789-+"
    super_ = "⁰¹²³⁴⁵⁶⁷⁸⁹⁻⁺"
    return str(exp).translate(str.maketrans(normal, super_))

def sci_notation(x):
    if np.isnan(x):
        return "–"
    if x == 0:
        return "0"
    exp = int(np.floor(np.log10(abs(x))))
    mant = x / 10**exp
    return f"{mant:05.2f}×10{superscript_exp(exp)}"

df_fmt = df.copy()
for col in ["runtime (s/pt)", "mean(error)", "RMS(error)"]:
    df_fmt[col] = df[col].apply(sci_notation)

# ---------------------------------------------------------------------
# 6. Compute relative speedup
# ---------------------------------------------------------------------
'''if "Julia_MP" in df["Code"].values:
    ref_speed = df.loc[df["Code"]=="Julia_MP", "runtime (s/pt)"].values[0]
    df_fmt["Speedup vs Julia_MP"] = df["runtime (s/pt)"].apply(
        lambda x: f"{ref_speed/x:5.2f}×" if not np.isnan(x) and x != 0 else "–"
    )'''

# ---------------------------------------------------------------------
# 7. Save results
# ---------------------------------------------------------------------
out_dir = os.path.join(root, "benchmark")
os.makedirs(out_dir, exist_ok=True)
out_txt = os.path.join(out_dir, "summary_table.txt")
out_csv = os.path.join(out_dir, "summary_table.csv")

with open(out_txt, "w", encoding="utf-8") as f:
    f.write("=== Gravitational Potential Accuracy & Speed Summary ===\n")
    f.write("Reference: Julia BigFloat (250 digits)\n")
    f.write("Julia 50 comparison performed at 52-digit arithmetic precision.\n\n")
    f.write(df_fmt.to_string(index=False))
    f.write("\n")

df_fmt.to_csv(out_csv, index=False, encoding="utf-8")

print(df_fmt)
print(f"\nSummary saved to:\n  {out_txt}\n  {out_csv}")


       Code runtime (s/pt)  mean(error)   RMS(error)
0    Python     01.49×10⁻⁴  09.97×10⁻¹⁷  01.30×10⁻¹⁶
1    MATLAB     03.39×10⁻⁵  02.66×10⁻¹⁶  03.18×10⁻¹⁶
2  Julia_MP     01.14×10⁻⁶  09.32×10⁻¹⁷  01.23×10⁻¹⁶
3  Julia_50     01.82×10⁻⁴  03.90×10⁻⁵¹  04.89×10⁻⁵¹

Summary saved to:
  /Volumes/Dunendran/Programs/Banchmark/Tetrahedron/GravPotentialBenchmark/benchmark/summary_table.txt
  /Volumes/Dunendran/Programs/Banchmark/Tetrahedron/GravPotentialBenchmark/benchmark/summary_table.csv
